# 4. Stitch outputs together
- Once you have fitted the weak, medium and dense diads, stitch them all together, and apply the Ne correction model

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import DiadFit as pf
import os
pf.__version__

'0.0.58'

In [2]:
MasterFolder=r"C:\Users\charl\Documents\Instruments and calibrations"
DayFolder=MasterFolder + "/UCB_4-10-23"

if not os.path.exists(MasterFolder + '\OUTPUT'):
    os.mkdir(MasterFolder +'\OUTPUT')
if not os.path.exists(MasterFolder + '\OUTPUT'+'\Pickles'):
    os.mkdir(MasterFolder + '\OUTPUT'+'\Pickles')

if not os.path.exists(DayFolder + '\GROUPS'):
    os.mkdir(DayFolder +'\GROUPS')

if not os.path.exists(DayFolder + '\FITOUT'):
    os.mkdir(DayFolder +'\FITOUT')


meta_path=DayFolder +  '\METADATA'
spectra_path=DayFolder + '\SPECTRA'

output_path=MasterFolder +"\OUTPUT"
fitout_path=DayFolder+'\FITOUT'
group_path=DayFolder+'\GROUPS'

## Load in specra, and combine

In [3]:
import os.path
from os import path

if os.path.exists(fitout_path+'/'+'Discarded_df.xlsx'):
    discard=pd.read_excel(fitout_path+'/'+'Discarded_df.xlsx')
else:
    discard=None
if os.path.exists(fitout_path+'/'+'Weak_Diads.xlsx'):
    grp1=pd.read_excel(fitout_path+'/'+'Weak_Diads.xlsx')
else:
    grp1=None
if os.path.exists(fitout_path+'/'+'Medium_Diads.xlsx'):
    grp2=pd.read_excel(fitout_path+'/'+'Medium_Diads.xlsx')
else:
    grp2=None
if os.path.exists(fitout_path+'/'+'Strong_Diads.xlsx'):
    grp3=pd.read_excel(fitout_path+'/'+'Strong_Diads.xlsx')
else:
    grp3=None

df2=pd.concat([grp1, grp2, grp3], axis=0).reset_index(drop=True).drop(columns=['Unnamed: 0'])

if discard is not None:
    discard_cols=discard[discard.columns.intersection(df2.columns)]
    df2=pd.concat([df2, discard_cols])

In [4]:
df2.head()

,filename,Splitting,Diad1_Combofit_Cent,Diad1_Combofit_Height,Diad1_Voigt_Cent,Diad1_Voigt_Area,Diad1_Voigt_Sigma,Diad1_Residual,Diad1_Prop_Lor,Diad1_fwhm,...,HB2_Sigma,C13_Cent,C13_Area,C13_Sigma,Diad2_Gauss_Cent,Diad2_Gauss_Area,Diad2_Gauss_Sigma,Diad1_Gauss_Cent,Diad1_Gauss_Area,Diad1_Gauss_Sigma
0,FDCA-P26-1,104.545351,1282.504157,367.496362,1282.504157,1052.927596,1.024066,3.267903,0.746348,2.048132,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,FDCA-P53-1,103.724480,1284.258669,1326.523430,1284.258669,3218.620131,0.896086,10.574944,0.666599,1.792173,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,FDCA-P54-1,103.706735,1284.314711,1241.314871,1284.314711,2984.081547,0.892089,9.241941,0.654681,1.784178,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,FDCA-P56-2,103.668123,1284.424338,1071.698051,1284.424338,2500.257715,0.859701,7.989505,0.671815,1.719403,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,FDCA-P57-2,103.626243,1284.511355,1295.911503,1284.511355,2890.272428,0.827629,9.110221,0.654708,1.655258,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## First, get the metadata for this day

In [5]:
# Put the common string you can in all your Ne lines here
ID_str='_'
file_ext_meta='txt' # Will only take files of this type 
diad_meta=pf.get_files(path=meta_path,
file_ext='txt', exclude_str=['N', 'FG', 'Cap','window','IMG','nodiad'],
 sort=True)
diad_meta

['FDCA-P1-1.txt',
 'FDCA-P1-2.txt',
 'FDCA-P1-3.txt',
 'FDCA-P10-1.txt',
 'FDCA-P10-2.txt',
 'FDCA-P10-3.txt',
 'FDCA-P11-1.txt',
 'FDCA-P12-1.txt',
 'FDCA-P13-1.txt',
 'FDCA-P14-1.txt',
 'FDCA-P14-2.txt',
 'FDCA-P14-3.txt',
 'FDCA-P15-1.txt',
 'FDCA-P16-1.txt',
 'FDCA-P17-1.txt',
 'FDCA-P18-1.txt',
 'FDCA-P18-2.txt',
 'FDCA-P19-1.txt',
 'FDCA-P19-2.txt',
 'FDCA-P19-3.txt',
 'FDCA-P2-1.txt',
 'FDCA-P2-2.txt',
 'FDCA-P2-3.txt',
 'FDCA-P20-1.txt',
 'FDCA-P21-1.txt',
 'FDCA-P22-1.txt',
 'FDCA-P23-1.txt',
 'FDCA-P23-2.txt',
 'FDCA-P23-3.txt',
 'FDCA-P24-1.txt',
 'FDCA-P25-1.txt',
 'FDCA-P26-1.txt',
 'FDCA-P26-2.txt',
 'FDCA-P27-1.txt',
 'FDCA-P27-2.txt',
 'FDCA-P27-3.txt',
 'FDCA-P28-1.txt',
 'FDCA-P29-1.txt',
 'FDCA-P3-1.txt',
 'FDCA-P3-2.txt',
 'FDCA-P3-3.txt',
 'FDCA-P30-1.txt',
 'FDCA-P31-1.txt',
 'FDCA-P31-2.txt',
 'FDCA-P31-3.txt',
 'FDCA-P32-1.txt',
 'FDCA-P33-1.txt',
 'FDCA-P34-1.txt',
 'FDCA-P35-1.txt',
 'FDCA-P35-2.txt',
 'FDCA-P35-3.txt',
 'FDCA-P36-1.txt',
 'FDCA-P36-2.txt',
 '

## Now get all the important information out of the metadata files

In [6]:
meta=pf.stitch_metadata_in_loop_witec(path=meta_path, 
Allfiles=diad_meta, prefix=False,
trupower=True)
meta['filename'].iloc[0]

100%|███████████████████████████████████████████████████████████████████████████████| 156/156 [00:00<00:00, 247.92it/s]

Done
Oof, try not to work after midnight!


'FDCA-P45-1.txt'

## Then get a simple file name you can stitch with the spectra

In [7]:
file_m=pf.extracting_filenames_generic(names=meta['filename'],
    prefix=False, str_prefix=" ",
   file_type='.txt')
# Checks they are all unique
file_m[0]

good job, no duplicate file names


'FDCA-P45-1'

## Now get filename from spectra into same form as metadata

In [8]:
# Remove these to get the pure file name
file_s=pf.extracting_filenames_generic(
    prefix=False, str_prefix=" ",
    names=df2['filename'].reset_index(drop=True),
   file_type='.txt')

# Remove the cosmic rays, as doesnt have it in the metatdata
file_s = np.char.replace(file_s.astype(str), "_CRR_DiadFit", "")

good job, no duplicate file names


In [9]:
file_s

array(['FDCA-P26-1', 'FDCA-P53-1', 'FDCA-P54-1', 'FDCA-P56-2',
       'FDCA-P57-2', 'FDCA-P58-1', 'FDCA-P59-1', 'FDCA-P60-1',
       'FDCA-P60-2', 'FDCA-P60-3', 'FDCA-P61-1', 'FDCA-P61-2',
       'FDCA-P61-3', 'FDCA-P62-1', 'FDCA-P62-2', 'FDCA-P62-3',
       'FDCA-P63-1', 'FDCA-P63-2', 'FDCA-P63-3', 'FDCA-P64-1',
       'FDCA-P64-2', 'FDCA-P64-3', 'FDCA-P65-1', 'FDCA-P65-2',
       'FDCA-P65-3', 'FDCA-P66-1', 'FDCA-P66-2', 'FDCA-P66-3',
       'FDCA-P67-1', 'FDCA-P67-2', 'FDCA-P67-3', 'FDCA-P68-1',
       'FDCA-P68-2', 'FDCA-P68-3', 'FDCA-P69-1', 'FDCA-P69-2',
       'FDCA-P69-3', 'FDCA-P70-1', 'FDCA-P70-2', 'FDCA-P70-3',
       'FDCA-P71-1', 'FDCA-P71-2', 'FDCA-P71-3', 'FDCA-P72-1',
       'FDCA-P72-2', 'FDCA-P72-3', 'FDCA-P73-1', 'FDCA-P73-2',
       'FDCA-P73-3', 'FDCA-P74-1', 'FDCA-P74-2', 'FDCA-P74-3',
       'FDCA-P75-1', 'FDCA-P75-2', 'FDCA-P75-3', 'FDCA-P76-1',
       'FDCA-P76-2', 'FDCA-P76-3', 'FDCA-P25-1', 'FDCA-P26-2',
       'FDCA-P27-1', 'FDCA-P27-2', 'FDCA-P27-3', 'FDCA-

In [10]:
# Combining them together
meta['name_for_matching']=file_m
df2['name_for_matching']=file_s
df2['Name_for_Secondary_Phases']=file_s
df_combo=df2.merge(meta, on='name_for_matching')

In [11]:
df_combo

,filename_x,Splitting,Diad1_Combofit_Cent,Diad1_Combofit_Height,Diad1_Voigt_Cent,Diad1_Voigt_Area,Diad1_Voigt_Sigma,Diad1_Residual,Diad1_Prop_Lor,Diad1_fwhm,...,Month,Day,power (mW),Int_time (s),accumulations,Mag (X),duration,24hr_time,sec since midnight,Spectral Center
0,FDCA-P26-1,104.545351,1282.504157,367.496362,1282.504157,1052.927596,1.024066,3.267903,0.746348,2.048132,...,April,10.0,35.089,20.0,5.0,5.0,"['0h', '1m', '42s']",8:14:34 PM\n,72874.0,1325.004
1,FDCA-P53-1,103.724480,1284.258669,1326.523430,1284.258669,3218.620131,0.896086,10.574944,0.666599,1.792173,...,April,11.0,34.653,20.0,5.0,5.0,"['0h', '1m', '42s']",1:23:33 AM\n,91413.0,1325.004
2,FDCA-P54-1,103.706735,1284.314711,1241.314871,1284.314711,2984.081547,0.892089,9.241941,0.654681,1.784178,...,April,11.0,34.703,20.0,5.0,5.0,"['0h', '1m', '41s']",1:30:06 AM\n,91806.0,1325.004
3,FDCA-P56-2,103.668123,1284.424338,1071.698051,1284.424338,2500.257715,0.859701,7.989505,0.671815,1.719403,...,April,11.0,34.668,20.0,5.0,5.0,"['0h', '1m', '41s']",1:50:00 AM\n,93000.0,1325.004
4,FDCA-P57-2,103.626243,1284.511355,1295.911503,1284.511355,2890.272428,0.827629,9.110221,0.654708,1.655258,...,April,11.0,34.660,20.0,5.0,5.0,"['0h', '1m', '42s']",2:01:13 AM\n,93673.0,1325.004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,FDCA-P8-2,105.226970,1281.208853,5252.652310,1281.208903,13963.088160,0.924072,5.704611,0.830558,1.848145,...,April,10.0,35.170,20.0,5.0,5.0,"['0h', '1m', '42s']",5:18:12 PM\n,62292.0,1325.004
152,FDCA-P8-3_CRR_DiadFit,105.224035,1281.213667,5480.071312,1281.213717,14613.013359,0.929288,5.603359,0.825469,1.858576,...,April,10.0,35.177,20.0,5.0,5.0,"['0h', '1m', '42s']",5:23:38 PM\n,62618.0,1325.004
153,FDCA-P9-1,105.183404,1281.285411,4742.816564,1281.285461,12515.816226,0.928378,5.132174,0.804832,1.856755,...,April,10.0,35.171,20.0,5.0,5.0,"['0h', '1m', '42s']",5:28:46 PM\n,62926.0,1325.004
154,FDCA-P9-2,105.175044,1281.285049,4898.570595,1281.285099,13077.659622,0.940507,5.779879,0.803712,1.881013,...,April,10.0,35.140,20.0,5.0,5.0,"['0h', '1m', '42s']",5:34:25 PM\n,63265.0,1325.004


In [14]:
from pickle import load
import pickle
import os
nm=os.path.basename(DayFolder)

pickle_path=MasterFolder + '\OUTPUT'+'\Pickles'
file_pkl=pickle_path + '/' +'Ne_lineReg_'+nm+'_PVM.pck'
with open(file_pkl, 'rb') as file_handle:
    my_loaded_model = pickle.load(file_handle)

In [15]:
df_combo['Ne_Correction']=my_loaded_model(df_combo['sec since midnight'])
df_combo.insert(1,'Corrected_Splitting', df_combo['Ne_Correction']*df_combo['Splitting'])

In [16]:
nm=os.path.basename(DayFolder)
#df_combo.to_excel(output_path+'/'+nm+'_Calibration_fitting.xlsx')

## If you have secondary phases, now is the time to merge those in

In [17]:
if os.path.exists(fitout_path+'/'+'Carb_Peak_fits.xlsx'):
    Carb=pd.read_excel(fitout_path+'/'+'Carb_Peak_fits.xlsx')
else:
    Carb=pd.DataFrame([])
if os.path.exists(fitout_path+'/'+'SO2_Peak_fits.xlsx'):
    SO2=pd.read_excel(fitout_path+'/'+'SO2_Peak_fits.xlsx')
else:
    SO2=pd.DataFrame([])

if (Carb.empty==False) & (SO2.empty==False):
    Sec_Phases=pd.merge(SO2, Carb, on='filename', how='outer')
    
if (Carb.empty==True) & (SO2.empty==False):
    Sec_Phases=SO2
if (Carb.empty==False) & (SO2.empty==True):
    Sec_Phases=Carb
if (Carb.empty==True) & (SO2.empty==True):
    Sec_Phases=pd.DataFrame([]) 
    print('You have no secondary phases!')

if Sec_Phases.empty==False:
    display(Sec_Phases.head())    

You have no secondary phases!


In [18]:
# Remove these to get the pure file name
if Sec_Phases.empty==False:
    file_sec_phase=pf.extracting_filenames_generic(
        prefix=False, str_prefix=" ",
        names=Sec_Phases['filename'].reset_index(drop=True),
       file_type='.txt')

    display(file_sec_phase)

In [19]:
if Sec_Phases.empty==False:
    Sec_Phases['name_for_matching']=file_sec_phase
    df_combo['name_for_matching']=df_combo['Name_for_Secondary_Phases']
    df_combo_sec_phase=df_combo.merge(Sec_Phases, on='name_for_matching', how='outer')
else:
    df_combo_sec_phase=df_combo

### Add some more useful columns

In [20]:
if 'Peak_Area_Carb' in df_combo_sec_phase.columns:
    df_combo_sec_phase['Carb_Diad_Ratio']=(df_combo_sec_phase['Peak_Area_Carb']/(df_combo_sec_phase['Diad1_Voigt_Area']
                      +df_combo_sec_phase['Diad2_Voigt_Area']))
if 'Peak_Area_SO2' in df_combo_sec_phase.columns:
    df_combo_sec_phase['SO2_Diad_Ratio']=(df_combo_sec_phase['Peak_Area_SO2']/(df_combo_sec_phase['Diad1_Voigt_Area']
                      +df_combo_sec_phase['Diad2_Voigt_Area']))

In [21]:
df_combo_sec_phase.to_excel(output_path+'/'+nm+'_Calibration_fitting_w_sec_pg.xlsx')
os.system('echo -en "\a"')
print('done')

done
